In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_csv('Zip_Zri_MultiFamilyResidenceRental-2.csv')
census_data= pd.read_csv('acs2018.csv')
zillow_yvalues_data=pd.merge(df.iloc[:,0:8], df.iloc[:,8:].interpolate(axis=1), left_index=True, right_index=True)
date='2020-01'

In [3]:
def dataclean(zillow_yvalues_data,census_data,date):
    # takes in the zillow data with all the years and acs census data of two years before 
    # year to predict and date formatted '2020-01' and joins the census on zillow and cleans 
    
    
    zillow_yvalues_data_yearsubsect=zillow_yvalues_data[['RegionName','City','State',date]]
    fulldata=zillow_yvalues_data_yearsubsect.merge(census_data,how='left',left_on='RegionName', right_on='geo_id' ).drop(['geo_id'],axis=1)
    fulldata=fulldata[fulldata[date].isna()!=1]
    #drop column if >20% are nas 
    cols_to_drop=['do_date']
    for i in fulldata.columns:

        if sum(fulldata[i].isna())/len(fulldata)> .2:
            cols_to_drop.append(i)
   
    fulldata=fulldata.drop(cols_to_drop,axis=1)
    #drop rows with more than 50% nas
    #in this jan2020was one row
    fulldata=fulldata.loc[fulldata.isna().sum(axis=1)<.5*len(fulldata.columns)]
    #num of missing values in columns
    countofcols=0
    cols_with_na=[]
    for i in fulldata.columns:
        if sum(fulldata[i].isna())>0:
#             print('Column:',i)
#             print('Num of NAs:',sum(fulldata[i].isna()))
#             print('Num of 0s:',sum(fulldata[i]==0))
            cols_with_na.append(i)
            countofcols+=1
    #fill in nas with avrg of the column for the city 
    for i in cols_with_na:
        for indexofna in (list(fulldata[i][fulldata[i].isna()].index)):

            if bool(~(fulldata[fulldata.City==fulldata.loc[indexofna,'City']][i].mean()==fulldata[fulldata.City==fulldata.loc[indexofna,'City']][i].mean())):
                fulldata.loc[indexofna,i]=fulldata[fulldata.State==fulldata.loc[indexofna,'State']][i].mean()
                
            else:
                fulldata.loc[indexofna,i]=fulldata[fulldata.City==fulldata.loc[indexofna,'City']][i].mean()
                
    
    return fulldata

In [4]:
fulldata=dataclean(zillow_yvalues_data,census_data,date)

In [9]:
#train test split
tdata_train, tdata_test = train_test_split(fulldata, test_size=0.33, random_state=100)

In [10]:
tdata_train.to_csv('prelim_data_acsTRAIN.csv')
tdata_test.to_csv('prelim_data_acsTEST.csv')

In [11]:
fulldata


,RegionName,City,State,2020-01,total_pop,households,male_pop,female_pop,median_age,male_under_5,...,occupation_production_transportation_material,occupation_sales_office,occupation_services,management_business_sci_arts_employed,sales_office_employed,in_grades_1_to_4,in_grades_5_to_8,in_grades_9_to_12,in_school,in_undergrad_college
0,10025,New York,NY,3676.0,92805.0,41910.0,41890.0,50915.0,41.3,1811.0,...,1768.0,7237.0,5557.0,32166.0,7237.0,2694.0,3403.0,2356.0,22134.0,6152.0
1,60657,Chicago,IL,1731.0,70052.0,36624.0,35414.0,34638.0,31.2,1978.0,...,1525.0,10249.0,4323.0,33882.0,10249.0,1552.0,1591.0,812.0,12522.0,3196.0
2,10023,New York,NY,3561.0,62435.0,33693.0,27151.0,35284.0,43.3,1583.0,...,828.0,6925.0,1922.0,25796.0,6925.0,1429.0,1782.0,1069.0,10397.0,2595.0
3,77494,Katy,TX,1286.0,114989.0,35342.0,55688.0,59301.0,34.9,4326.0,...,1980.0,9323.0,4805.0,34386.0,9323.0,9079.0,9539.0,8093.0,39508.0,5843.0
4,60614,Chicago,IL,1916.0,71308.0,33584.0,35945.0,35363.0,30.3,2157.0,...,1136.0,10501.0,3125.0,32035.0,10501.0,2140.0,1853.0,1448.0,15963.0,5459.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1856,75202,Dallas,TX,1739.0,2059.0,1344.0,1130.0,929.0,32.2,0.0,...,35.0,315.0,84.0,1164.0,315.0,15.0,20.0,44.0,246.0,36.0
1857,84627,Ephraim,UT,1298.0,7357.0,1973.0,3376.0,3981.0,21.9,298.0,...,417.0,700.0,467.0,1200.0,700.0,279.0,430.0,493.0,3385.0,1816.0
1858,60301,Oak Park,IL,1625.0,2536.0,1595.0,1142.0,1394.0,45.3,31.0,...,30.0,279.0,38.0,1067.0,279.0,2.0,43.0,121.0,447.0,93.0
1859,84647,Mount Pleasant,UT,1221.0,4403.0,1485.0,2462.0,1941.0,37.6,167.0,...,334.0,343.0,331.0,515.0,343.0,201.0,225.0,327.0,1154.0,231.0


In [ ]:
rent_y_val=fulldata['2020-01']
rent_xvals=fulldata.drop(['2020-01','RegionName', 'City', 'State'],axis=1)

In [ ]:
# def get_yVals(df, date):
#     '''
#     Enter in your dataframe for Zillow Rent Index, enter in your desired month/year combo for the rent values per zip code 
#     in the for 'YYYY-MM'. The output is a pandas Series with the zip codes as indexes and the rent index as its values.
#     '''
#     return pd.merge(df.iloc[:,0:8], df.iloc[:,8:].interpolate(axis=1), left_index=True, right_index=True).set_index('RegionName')[date]

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Lasso
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

In [ ]:
#CV LASSO FOR REG DATA
cvlasso=LassoCV( alphas=np.linspace(1,1000,100), fit_intercept=True, normalize=False,cv=5)
cvlasso.fit(rent_xvals,rent_y_val)

lasso=Lasso(alpha=cvlasso.alpha_, fit_intercept=True, normalize=False)

print(f'The selected alpha was {cvlasso.alpha_} which gave r^2 of ')
print(cross_val_score(lasso, rent_xvals,rent_y_val,  scoring='r2'))